In [1]:
from collections import Counter, defaultdict
import operator
import stringdist
import os
import re
import pickle
from tqdm import tqdm

In [2]:
texts = []

# for i in os.listdir('data'):
#     for j in os.listdir(f'data/{i}'):
#         file = os.listdir(f'data/{i}/{j}')[0]
#         with open(f"data/{i}/{j}/{file}", encoding="utf-8") as f:
#             texts.append(f.read().replace(u'\xa0', ' '))
with open("data/all_news.pkl", 'rb') as f:
    news = pickle.load(f)
    
for doc in news:
    texts.append(doc['body'])

In [3]:
cnt = Counter()

In [4]:
words = []
pattern = "[а-яА-Я]{3,}"

for text in texts:
    text_words = text.split()
    for word in text_words:
        corrects = re.findall(pattern, word)
        if len(corrects) == 1:
            words.append(corrects[0].lower())

In [5]:
for word in words:
    cnt[word] += 1

In [6]:
def find_nearest(word, n=10):
    result = []
    for cnt_word, count in cnt.items():
        dist = stringdist.rdlevenshtein(word, cnt_word)
        result.append((cnt_word, dist, count))
        
    result.sort(key=operator.itemgetter(1, 2))
    
    return result[:n]

In [7]:
%%time

find_nearest('призидент')

Wall time: 27.1 s


[('президент', 1, 3354),
 ('резидент', 2, 15),
 ('президенте', 2, 139),
 ('президенты', 2, 153),
 ('президенту', 2, 380),
 ('президента', 2, 3562),
 ('причинено', 3, 1),
 ('президентка', 3, 1),
 ('пригоден', 3, 1),
 ('привидений', 3, 1)]

## Bigrams

In [8]:
bigram_index = defaultdict(list)

In [9]:
for word in set(words):
    bigrams = [word[i:i+2] for i in range(len(word)-1)]
    [bigram_index[bigram].append(word) for bigram in bigrams]

In [10]:
def find_nearest_by_index(word, n=10):
    result = []
    
    bigram_words = []
    bigrams = [word[i:i+2] for i in range(len(word)-1)]
    [bigram_words.extend(bigram_index[bigram]) for bigram in bigrams]
    
    c = Counter(bigram_words)
    bigram_set = {k for k,v in c.items() if v > 1 and len(k)-len(word) < 3 and len(k)-len(word) > -3}
    
    for bigram_word in bigram_set:
        dist = stringdist.rdlevenshtein(word, bigram_word)
        result.append((bigram_word, dist, cnt[bigram_word]))
        
    result.sort(key=operator.itemgetter(1, 2))
    
    return result[:n]

In [11]:
%%time

find_nearest_by_index('призидент')

Wall time: 1.05 s


[('президент', 1, 3354),
 ('резидент', 2, 15),
 ('президенте', 2, 139),
 ('президенты', 2, 153),
 ('президенту', 2, 380),
 ('президента', 2, 3562),
 ('пригоден', 3, 1),
 ('привидений', 3, 1),
 ('причинено', 3, 1),
 ('президентах', 3, 1)]